In [9]:
import numpy as np
def dec_to_bin(number: int, bits=-1):
    number=int(np.round(number, 0))
    neg=False
    out=""
    if (number<0):
        number*=-1
        number-=1
        neg=True
    while (number>0):
        res = number%2
        if (neg):
            res= 0 if (res==1) else 1
        out=f"{res}{out}"
        if (len(out)==(bits-1)):
            break
        number=int(number/2)
        
    if (neg):
        out=f"{1}{out}"
    else: out=f"{0}{out}"
    if (len(out)==0):
        out="0"
    while (len(out)<bits):
        out=f"{out[0]}{out}"
    return out
def bin_to_dec(number: str):
    output = 0
    neg = False
    if (number[0]=="1"):
        neg = True
    for i, char in enumerate(number[:0:-1]):
        if (neg):
            output+=(1-int(char))*(2**i)
        else:
            output+=int(char)*(2**i)
    if (neg):
        output*=-1
        output-=1
    return output

In [10]:
import numpy as np
test = np.load("X_test.npy")
test = test.flatten()
with open("X_test.txt", "w") as f:
    for num in test:
        num=num*(2**10)
        f.write(f"{dec_to_bin(num, 16)}\n")

In [74]:
num  = dec_to_bin(-0.037350*(2**10), 16)
print(num)
print(bin_to_dec(num))
print(bin_to_dec(num)*(2**-10))
print(-0.037350*(2**10))

1111111111011011
-37
-0.0361328125
-38.2464


In [ ]:
print(len(arr))

In [11]:
import numpy as np
arr = np.load("X_test.npy")
NFRAC = 10
WIDTH = 16
with open("X_test.txt", "w") as f:
    for num in arr.flatten():
        num=num*(2**NFRAC)
        bin_num = dec_to_bin(int(np.round(num, 0)), WIDTH)
        f.write(f"{bin_num}\n")

print(arr)

[[-0.11950312  0.40616292 -1.04058613 ...  0.40865762 -1.01995561
  -0.18016747]
 [ 0.30887296  0.2271202  -1.15605569 ...  1.86744644 -1.23200286
  -1.19496887]
 [-1.26550208  0.66753284  1.40874527 ...  0.58246973  1.3092331
   1.75718066]
 ...
 [ 1.14562686 -0.46692901 -0.36909324 ... -1.18354148  0.05067312
  -0.68756817]
 [-0.1047317   0.3005048   1.3434744  ... -0.52088167  1.38000935
   0.23497856]
 [-0.91237697  0.72960102  0.17986115 ... -0.29436679  0.09422757
   0.55786991]]


In [10]:
print(arr[0:10]*(2**10))

[[ -122.37119642   415.9108303  -1065.56019204  -844.52486746
   -773.46427742  -594.91674529  2034.60199599  1574.97625449
   2034.60199599   646.38750434   392.76695208  -206.13663678
   1085.20077818   418.46540268 -1044.43454586  -184.49149042]
 [  316.28591127   232.57108876 -1183.80102541  -866.38709617
  -1059.44498624  -634.71129353   187.60518529   610.14330009
    187.60518529  1743.22322184  2089.98050153  2480.90927304
   1561.03645411  1912.26515221 -1261.57093032 -1223.64812477]
 [-1295.87412549   683.55362615  1442.55515314  1434.81314935
   1615.39002959  1323.74501829   126.46960484    27.69471257
    126.46960484   336.47963032   533.09678763   750.94199603
    565.20946009   596.44900603  1340.65469158  1799.35299335]
 [-1090.62332059   698.72300217  2226.09560424  2287.76243372
   2099.86117439  2408.32315294  -136.09367774   233.76780265
   -136.09367774   928.4191793   1137.4213171   1651.43553693
    933.60060655  1429.31962458  1856.11275691   996.36832135]
 [-1

In [ ]:
ans = np.load("y_test.npy")
print(ans[0:10])